In [1]:
import numpy as np
import pickle as pkl

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib as mpl

import sys
sys.path.append("../functions/")

import preprocessing
import experiment_settings
import build_model
import metricplots
import allthelinalg
import analysisplots

import importlib as imp

# pretty plots
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.dpi'] = 150
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.size'] = 12
mpl.rcParams['font.sans-serif']=['Verdana']

params = {"ytick.color": "k",
          "xtick.color": "k",
          "axes.labelcolor": "k",
          "axes.edgecolor": "k"}
plt.rcParams.update(params)


In [2]:
source = "ERSST"

In [3]:
modelpath = "../models/"
experiment_name = "allcmodel-tos_allcmodel-tos_1-5yearlead_tvtfolds"
experiment_dict = experiment_settings.get_experiment_settings(experiment_name)

filefront = experiment_dict["filename"]
filename = modelpath + filefront
ntrainvariants = experiment_dict["ntrainvariants"]
nvalvariants = experiment_dict["nvalvariants"]
ntestvariants = experiment_dict["ntestvariants"]
foldseeds = experiment_dict["foldseeds"]
seedlist = experiment_dict["seeds"]
modellist = experiment_dict["modellist"]
outbounds = experiment_dict["outbounds"]
run = experiment_dict["run"]
leadtime = experiment_dict["leadtime"]

lon, lat = preprocessing.outlonxlat(experiment_dict)
nvars = int(nvalvariants*len(modellist))
centre = (outbounds[2]+outbounds[3])/2
nvariant = ntestvariants # check!
nmodels = len(modellist)
nmems = 30
obsyearvec = np.arange(1870+2*run+leadtime,2023,)

weights = np.meshgrid(lon,lat)[1]
latweights = np.sqrt(np.cos(np.deg2rad(weights)))

In [4]:
data_experiment_name = "allcmodel-tos_allcmodel-tos_1-5yearlead"
data_experiment_dict = experiment_settings.get_experiment_settings(data_experiment_name)
datafilefront = data_experiment_dict["filename"]
datafile = "../processed_data/" + datafilefront + ".npz"

datamat = np.load(datafile)

allinputdata = datamat["allinputdata"]
alloutputdata = datamat["alloutputdata"] 

In [5]:
def reshapengrab(inputmatrix,ind,nmodels,nvariants):

    # grabs all variants (nvariant) for a single climate model data (ind of nmodel) 
    inputshape = inputmatrix.shape
    ntimesteps = int(inputshape[0]/(nmodels*nvariants))
    if len(inputshape) == 3:        
        intmatrix = np.reshape(inputmatrix,(nmodels,nvariants,ntimesteps,inputshape[1],inputshape[2]))
    else:
        intmatrix = np.reshape(inputmatrix,(nmodels,nvariants,ntimesteps,inputshape[1],inputshape[2],inputshape[3]))
    
    outputint = intmatrix[ind]
    if len(inputshape) == 3:
        shapeout = (ntimesteps*nvariants,inputshape[1],inputshape[2])
    else:
        shapeout = (ntimesteps*nvariants,inputshape[1],inputshape[2],inputshape[3])
    outputmatrix = np.reshape(outputint,shapeout)    
    
    return outputmatrix

def model_member_corr(pattern,outputtest,y_pred_test,outputval,y_pred_val,nmodels,nvariant,landmask):
    
    modelpearsons = np.empty((nmodels,nvariant))
    modelps = np.empty((nmodels,nvariant))

    patterndims = len(pattern.shape)
    
    for imodel in range(nmodels):
             
        outputtestloop = reshapengrab(outputtest,imodel,nmodels,nvariant)    
        y_pred_testloop = reshapengrab(y_pred_test,imodel,nmodels,nvariant)    
        if patterndims == 2:            
            for imem in range(nvariant):
                outputtest_singlemem = outputtestloop[ntimesteps*imem:ntimesteps*(imem+1)]
                y_pred_test_singlemem = y_pred_testloop[ntimesteps*imem:ntimesteps*(imem+1)]
        
                modelpearsons[imodel,imem],modelps[imodel,imem] = allthelinalg.corr_indextimeseries(
                    pattern,outputtest_singlemem,y_pred_test_singlemem,outputval,y_pred_val,landmask)
        
        elif patterndims == 3:
            for imem in range(nvariant):
                outputtest_singlemem = outputtestloop[ntimesteps*imem:ntimesteps*(imem+1)]
                y_pred_test_singlemem = y_pred_testloop[ntimesteps*imem:ntimesteps*(imem+1)]
        
                modelpearsons[imodel,imem],modelps[imodel,imem] = allthelinalg.corr_indextimeseries(
                    pattern[imodel],outputtest_singlemem,y_pred_test_singlemem,outputval,y_pred_val,landmask)
    
    return modelpearsons

def weightedMSE(y_pred,y_true,weights):
    err = ((y_pred-y_true)*weights)
    sqerr = err**2
    mse = np.mean(sqerr)
    return mse

In [6]:
trainvaltestfile = "../processed_data/foldseeds" + filefront + ".pkl"
with open(trainvaltestfile,'rb') as f:
    trainvaltestmat = pkl.load(f)

In [18]:
# Go through each CNN from each fold and find which initialization has the best skill on validation data, then use that for the comparison for each val fold.

modelscorr = np.empty((len(foldseeds),len(seedlist),nmodels,nvariant))
obscorr = np.empty((len(foldseeds),len(seedlist)))

obstimeseries_true = np.empty((len(foldseeds),len(seedlist),len(obsyearvec)))
obstimeseries_pred = np.empty((len(foldseeds),len(seedlist),len(obsyearvec)))

valmse = np.empty((len(foldseeds),len(seedlist)))

for ifold, foldseed in enumerate(foldseeds):
    
    np.random.seed(foldseed)
    memorder = trainvaltestmat[ifold]
    
    trainvaltest = [
                    memorder[:ntrainvariants],
                    memorder[ntrainvariants:(ntrainvariants+nvalvariants)],
                    memorder[(ntrainvariants+nvalvariants):]
                    ]
    
    inputdata,inputval,inputtest,outputdata,outputval,outputtest = preprocessing.splitandflatten(
        allinputdata,alloutputdata,trainvaltest,experiment_dict["run"])
    
    inputdata[:, np.isnan(np.mean(inputdata, axis=0))] = 0
    inputval[:, np.isnan(np.mean(inputval, axis=0))] = 0
    inputtest[:, np.isnan(np.mean(inputtest, axis=0))] = 0
    
    outputstd = np.std(outputdata, axis=0, keepdims=True)
    outputdata = outputdata/outputstd
    outputval = outputval/outputstd
    outputtest = outputtest/outputstd
    
    outputdata[:, np.isnan(np.mean(outputdata, axis=0))] = 0
    outputval[:, np.isnan(np.mean(outputval, axis=0))] = 0
    outputtest[:, np.isnan(np.mean(outputtest, axis=0))] = 0 
    
    ntimesteps = int(len(outputval)/(nvariant*nmodels))
    landmask = (np.mean(outputval,axis=0))!=0

    inputobs,outputobs = preprocessing.make_inputoutput_obs(experiment_dict,source)
    inputobs,outputobs = preprocessing.concatobs(inputobs,outputobs,outputstd,run)

    for iseed,random_seed in enumerate(seedlist):
    
        fileout = filename + "_seed=" + str(random_seed) + "_foldseed_" + str(foldseed) +".h5"
        
        full_model = build_model.build_CNN_full(inputval, outputval, 
                                                        experiment_dict, random_seed)   
        
        full_model.compile(optimizer=tf.keras.optimizers.legacy.SGD(experiment_dict["learning_rate"]),  # optimizer
                           )
    
        full_model.load_weights(fileout)
        
        full_model.trainable = False # freeze BN
        
        y_pred_val = full_model.predict(inputval)
        y_pred_test = full_model.predict(inputtest,verbose=0)
        y_pred_obs = full_model.predict(inputobs,verbose=0)

        valmse[ifold,iseed] = weightedMSE(y_pred_val,outputval,latweights)
        
        bestpattern = allthelinalg.calculate_SC_weighted(y_pred_val,outputval,landmask,latweights)  

        modelscorr[ifold,iseed,:,:] = model_member_corr(bestpattern,outputtest,y_pred_test,outputval,y_pred_val,nmodels,nvariant,landmask)
        obscorr[ifold,iseed],_ = allthelinalg.corr_indextimeseries(bestpattern,outputobs,y_pred_obs,outputval,y_pred_val,landmask)

        obstimeseries_true[ifold,iseed,:] = allthelinalg.index_timeseries(outputobs,bestpattern,landmask)
        obstimeseries_pred[ifold,iseed,:] = allthelinalg.index_timeseries(y_pred_obs,bestpattern,landmask)
        

conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conversion done
336/336 [==============================] - 1s 3ms/step
conver

In [13]:
bestseeds = np.argmin(valmse,axis=1)

for ifold in range(len(foldseeds)):
    analysisplots.prettyscatterplot(modelscorr[ifold,bestseeds[ifold],:,:],[obscorr[ifold,bestseeds[ifold]],modellist,np.arange(ntestvariants),"correlation coefficient",[source],None)

NameError: name 'modelscorr' is not defined

In [9]:
obstimeseries_true.shape

(15, 3, 143)

In [10]:
import LIM

tau = 5

bigdatashape = alloutputdata.shape

trainvariants = np.arange(15)
valvariants = np.arange(15,23)
testvariants = np.arange(23,30)

traindims = LIM.get_partial_dims(alloutputdata,trainvariants)
valdims = LIM.get_partial_dims(alloutputdata,valvariants)
testdims = LIM.get_partial_dims(alloutputdata,testvariants)
obsyearvec2 = np.arange(1870+3*run+leadtime,2023,)


G = LIM.LIM_getG(outputdata,tau,traindims,landmask)

y_pred_val,outputval_tau = LIM.LIM_cmodel(outputval,tau,valdims,landmask,G)
y_pred_test,outputtest_tau = LIM.LIM_cmodel(outputtest,tau,testdims,landmask,G)

bestpattern = allthelinalg.calculate_SC_weighted(y_pred_val,outputval_tau,landmask,latweights)   

# analysisplots.plotpattern_SST(bestpattern,lon,lat,outputstd)

y_pred_obs_ERSST,outputobs_tau = LIM.LIM_obs(outputobs,tau,landmask,G)

nmode = LIM.get_normalmodes(G,latweights)

# analysisplots.plotpattern_SST(nmode,lon,lat,outputstd)

LIM.patternplots_SST(bestpattern,nmode,outputobs_tau,y_pred_obs_ERSST,outputval,y_pred_val,
                              landmask,lon,lat,obsyearvec2,source,outputstd)


TypeError: LIM_getG() missing 1 required positional argument: 'weights'

In [ ]:
avgobstrue = np.mean(obstimeseries_true,axis=(0,1))
avgobspred = np.mean(obstimeseries_pred,axis=(0,1))

maxobstrue = np.max(obstimeseries_true,axis=(0,1))
minobstrue = np.min(obstimeseries_true,axis=(0,1))

maxobspred = np.max(obstimeseries_pred,axis=(0,1))
minobspred = np.min(obstimeseries_pred,axis=(0,1))

nmodeindex = allthelinalg.index_timeseries(outputobs,nmode,landmask)
nmodepred = allthelinalg.index_timeseries(y_pred_obs_ERSST,nmode,landmask)

plt.figure(figsize=(12,6))

# for ifold in range(len(foldseeds)):
#     plt.plot(obsyearvec,np.transpose(obstimeseries_pred[ifold,:,:]),alpha=0.2)

#plt.fill_between(obsyearvec,minobstrue,maxobstrue,color='xkcd:mint green',alpha=0.5,label='true range')
plt.fill_between(obsyearvec,minobspred,maxobspred,color='xkcd:beige',alpha=0.5,label='pred range')

plt.plot(obsyearvec,avgobstrue,color='xkcd:teal',label="SC from CNN")
plt.plot(obsyearvec,avgobspred,color='xkcd:golden rod',label="mean CNN pred of SC")

plt.plot(obsyearvec,nmodeindex,color='#002fa7',label="Normal mode from LIM")
plt.plot(obsyearvec2,nmodepred,color='xkcd:plum',label="Normal mode predicted by LIM")

plt.xlabel("year")

plt.legend()
plt.tight_layout()

plt.show()

In [ ]:
y_pred_test = full_model.predict(inputtest)

SCindex_model_true = allthelinalg.index_timeseries(outputtest,bestpattern,landmask)
SCindex_model_pred = allthelinalg.index_timeseries(y_pred_test,bestpattern,landmask)
n1model = ntimesteps*ntestvariants
plt.figure(figsize=(10,10))

plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab10(np.linspace(0,1,nmodels)))

for imodel,cmodel in enumerate(modellist):

    SCindex_model_true_loop = SCindex_model_true[imodel*n1model:(imodel+1)*n1model]
    SCindex_model_pred_loop = SCindex_model_pred[imodel*n1model:(imodel+1)*n1model]
    
    plt.subplot(3,3,imodel+1)
    for ivariant in range(nvariant):
        plt.scatter(SCindex_model_true_loop[ivariant*ntimesteps:(ivariant+1)*ntimesteps],
                    SCindex_model_pred_loop[ivariant*ntimesteps:(ivariant+1)*ntimesteps],marker='.')
    plt.title(cmodel)
    plt.plot(np.arange(-3,4),np.arange(-3,4),color='xkcd:slate gray')
    plt.ylim(-4.5,4.5)
    plt.xlim(-4.5,4.5)


plt.tight_layout()

In [ ]:
SCindex_model_pred_loop[ivariant*ntimesteps:(ivariant+1)*ntimesteps]

In [ ]:
nvariant